In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
products_df = pd.read_csv('./Data/products.csv')
products_df.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level
0,1001,Organic Rice,Groceries,1551.26,2296.31,43
1,1002,Olive Oil,Groceries,2516.38,3370.60,39
2,1003,Canned Beans,Groceries,961.25,1117.36,30
3,1004,Pasta,Groceries,696.83,1007.94,36
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27


In [6]:
sales_df = pd.read_csv('./Data/sales.csv')
sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date']).dt.date
sales_df.head()

,product_id,quantity_sold,sale_date,location,discount
0,1013,4,2024-09-02,Rajshahi,52.178882
1,1064,9,2024-02-25,Sylhet,52.059982
2,1011,2,2024-02-07,Dhaka,7.370982
3,1019,2,2024-09-01,Sylhet,51.219448
4,1011,9,2024-01-21,Chittagong,82.274403


In [7]:
from datetime import datetime

date1 = datetime.strptime('2024-09-01', '%Y-%m-%d').date()
date2 = datetime.strptime('2024-09-03', '%Y-%m-%d').date()
date = sales_df.iloc[0]['sale_date']

date1 <= date <= date2 

True

In [8]:
purchases_df = pd.read_csv('./Data/purchases.csv')
purchases_df['purchase_date'] = pd.to_datetime(purchases_df['purchase_date']).dt.date
purchases_df.head()

,product_id,quantity_purchased,purchase_date
0,1095,94,2024-08-19
1,1054,23,2024-08-16
2,1058,64,2024-12-18
3,1067,57,2024-06-12
4,1046,91,2024-03-13


### 1. Compute current stock

In [9]:
product_id = 1095
quantity_purchased = purchases_df[
    purchases_df['product_id'] == product_id]['quantity_purchased'
].sum()
quantity_sold = sales_df[sales_df['product_id'] == 1095]['quantity_sold'].sum()
current_stock = quantity_purchased - quantity_sold
print(current_stock)

476


In [10]:
def get_current_stock(product_id):
    quantity_purchased = purchases_df[purchases_df['product_id'] == product_id]['quantity_purchased'].sum()
    quantity_sold = sales_df[sales_df['product_id'] == product_id]['quantity_sold'].sum()
    current_stock = quantity_purchased - quantity_sold
    return current_stock

In [11]:
print(get_current_stock(1095))

476


In [12]:
products_df['current_stock'] = products_df['product_id'].apply(
    lambda product_id: get_current_stock(product_id)
)

In [13]:
products_df.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120
3,1004,Pasta,Groceries,696.83,1007.94,36,204
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575


### 2. Compute profit per product

In [18]:
def get_profit(product_id):
    quantity_sold = sales_df[sales_df['product_id'] == product_id]['quantity_sold'].sum()
    product = products_df[products_df['product_id'] == product_id]
    profit_per_sale = product['selling_price'] - product['cost_price']
    profit_per_sale = profit_per_sale.iloc[0]
    total_profit = profit_per_sale * quantity_sold
    return total_profit

products_df['profit'] = products_df['product_id'].apply(
    lambda product_id: get_profit(product_id)
)

products_df.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock,profit
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658,158695.65
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301,146071.62
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120,33407.54
3,1004,Pasta,Groceries,696.83,1007.94,36,204,101732.97
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575,134286.66


### 3. Find the slow moving products

In [19]:
start_date = datetime.strptime('2024-12-31', '%Y-%m-%d').date()
cutoff_date = start_date - timedelta(days=90)
print(cutoff_date)

2024-10-02


In [20]:
def is_slow_moving(product_id):
    start_date = datetime.strptime('2024-12-31', '%Y-%m-%d').date()
    cutoff_date = start_date - timedelta(days=90)
    last_90_days_sales = sales_df[
        (sales_df['product_id'] == product_id) &
        (sales_df['sale_date'] >= cutoff_date)
    ]
    total_recent_sales = last_90_days_sales['quantity_sold'].sum()
    return total_recent_sales < 40

products_df['slow_moving'] = products_df['product_id'].apply(is_slow_moving)
products_df.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock,profit,slow_moving
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658,158695.65,False
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301,146071.62,True
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120,33407.54,False
3,1004,Pasta,Groceries,696.83,1007.94,36,204,101732.97,False
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575,134286.66,True


In [21]:
products_df['slow_moving'].value_counts()

slow_moving
False    88
True     12
Name: count, dtype: int64

### 4. Flag overstocked and understocked products

In [46]:
def get_stock_status(product_id):
    product = products_df[products_df['product_id'] == product_id].iloc[0]
    stock = product['current_stock']
    reorder = product['reorder_level']
    if stock < reorder: return 'Understocked'
    elif stock > reorder * 15: return 'Overstocked'
    return 'Properly Stocked'

products_df['stock_status'] = products_df['product_id'].apply(get_stock_status)
products_df.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock,profit,slow_moving,stock_status
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658,158695.65,False,Overstocked
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301,146071.62,True,Properly Stocked
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120,33407.54,False,Properly Stocked
3,1004,Pasta,Groceries,696.83,1007.94,36,204,101732.97,False,Properly Stocked
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575,134286.66,True,Overstocked


In [23]:
products_df['stock_status'].value_counts()

stock_status
Properly Stocked    72
Overstocked         17
Understocked        11
Name: count, dtype: int64

### 5. Get Reveneu per product

In [24]:
def get_revenue(products_df, sales_df, product_id):
    selling_price = products_df[
        products_df['product_id'] == product_id
    ]['selling_price'].iloc[0]
    
    quantity_sold = sales_df[
        sales_df['product_id'] == product_id
    ]['quantity_sold'].sum()
    
    revenue = selling_price * quantity_sold
    return revenue

### 6. Queries business data

In [25]:
def get_sales_between_dates(sales_df, start_date, end_date, locations):
    return sales_df[
        (sales_df['sale_date'] >= start_date)
        & (sales_df['sale_date'] <= end_date) 
        & (sales_df['location'].isin(locations))
    ]

In [26]:
def get_products_of_selected_categories(products_df, categories):
    return products_df[products_df['category'].isin(categories)]

In [27]:
def get_under_stocked_products(products_df):
    return products_df[products_df['stock_status'] == 'Understocked']

In [28]:
def get_summary_kpis(products_df, sales_df):
    total_revenue = products_df['product_id'].apply(
        lambda product_id: get_revenue(products_df, sales_df, product_id)
    ).sum()
    
    total_profit = products_df['profit'].sum()
    total_units_sold = sales_df['quantity_sold'].sum()
    total_understocked_products = len(get_under_stocked_products(products_df))
    return {
        'Total Revenue (K)': int(total_revenue/1e3),
        'Total Profit (K)': int(total_profit/1e3),
        'Total Units Sold (K)': int(total_units_sold/1e3),
        'Total Understocked Products': total_understocked_products
    }

In [29]:
products_df['category'].unique()

array(['Groceries', 'Electronics', 'Clothing', 'Perishables'],
      dtype=object)

In [30]:
start_date = datetime.strptime('2024-01-01', '%Y-%m-%d').date()
end_date = datetime.strptime('2024-12-31', '%Y-%m-%d').date()
selected_locations = ['Dhaka']
selected_categories = ['Groceries']

In [31]:
filtered_sales = get_sales_between_dates(
    sales_df,
    start_date, 
    end_date, 
    selected_locations
)

filtered_sales.head()

,product_id,quantity_sold,sale_date,location,discount
2,1011,2,2024-02-07,Dhaka,7.370982
5,1077,7,2024-04-19,Dhaka,94.679928
8,1001,6,2024-08-18,Dhaka,56.834725
9,1040,7,2024-01-08,Dhaka,89.031249
11,1076,6,2024-10-16,Dhaka,96.526187


In [32]:
filtered_products = get_products_of_selected_categories(
    products_df,
    selected_categories
)

filtered_products.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock,profit,slow_moving,stock_status
0,1001,Organic Rice,Groceries,1551.26,2296.31,43,658,158695.65,False,Overstocked
1,1002,Olive Oil,Groceries,2516.38,3370.60,39,301,146071.62,True,Properly Stocked
2,1003,Canned Beans,Groceries,961.25,1117.36,30,120,33407.54,False,Properly Stocked
3,1004,Pasta,Groceries,696.83,1007.94,36,204,101732.97,False,Properly Stocked
4,1005,Coffee Beans,Groceries,2163.01,2991.94,27,575,134286.66,True,Overstocked


In [33]:
understocked_products = get_under_stocked_products(
    filtered_products
)
understocked_products.head()

,product_id,product_name,category,cost_price,selling_price,reorder_level,current_stock,profit,slow_moving,stock_status
47,1048,Almond Milk,Groceries,2621.61,3401.59,33,-47,180955.36,False,Understocked
80,1081,Organic Rice,Groceries,1533.01,2074.05,22,13,153655.36,False,Understocked
81,1082,Olive Oil,Groceries,2250.53,2957.89,47,5,200182.88,False,Understocked


### Summary KPIs

In [34]:
get_summary_kpis(filtered_products, filtered_sales)

{'Total Revenue (K)': 4102,
 'Total Profit (K)': 3543,
 'Total Units Sold (K)': 6,
 'Total Understocked Products': 3}

In [35]:
products_df.columns

Index(['product_id', 'product_name', 'category', 'cost_price', 'selling_price',
       'reorder_level', 'current_stock', 'profit', 'slow_moving',
       'stock_status'],
      dtype='object')

In [36]:
sales_df.columns

Index(['product_id', 'quantity_sold', 'sale_date', 'location', 'discount'], dtype='object')

In [37]:
purchases_df.columns

Index(['product_id', 'quantity_purchased', 'purchase_date'], dtype='object')

# Visualization using Plotly

In [42]:
import plotly.express as px
top_products = filtered_products.nlargest(10, 'profit')[['product_name', 'profit']]
plot1 = px.bar(top_products, x='product_name', y='profit', title="Top 10 Products by Profit")

    product_name     profit
44  Coffee Beans  246773.82
46  Granola Bars  241602.34
81     Olive Oil  200182.88
43         Pasta  197819.76
40  Organic Rice  192514.56
47   Almond Milk  180955.36
0   Organic Rice  158695.65
80  Organic Rice  153655.36
1      Olive Oil  146071.62
48         Honey  142789.50


In [43]:
plot1.show()

In [44]:
category_profit = filtered_products.groupby('category')['profit'].sum().reset_index()
plot2 = px.pie(category_profit, values='profit', names='category', title="Profit Distribution by Category")
plot2.show()

# Tables

In [41]:
summary_df = filtered_products[['product_name', 'category', 'current_stock', 'reorder_level', 'profit', 'stock_status']]
summary_df.head()

,product_name,category,current_stock,reorder_level,profit,stock_status
0,Organic Rice,Groceries,658,43,158695.65,Overstocked
1,Olive Oil,Groceries,301,39,146071.62,Properly Stocked
2,Canned Beans,Groceries,120,30,33407.54,Properly Stocked
3,Pasta,Groceries,204,36,101732.97,Properly Stocked
4,Coffee Beans,Groceries,575,27,134286.66,Overstocked
